# Zusammenführen der OSM-Datensätze ohne `service` (plus SimRa-Datensatz)

In [2]:
import geopandas as gpd

In [3]:
# Lade der GeoJSON-Dateien
maxspeed = gpd.read_file("../../data/processed_data/osm_maxspeed_max_noservice.geojson")
surface = gpd.read_file("../../data/processed_data/osm_surface_ratios_noservice.geojson")
highway = gpd.read_file("../../data/processed_data/osm_highway_without_service_ratios.geojson")

In [5]:
maxspeed.head(3)

,id,type,score,incidents,rides,index_right,maxspeed,geometry
0,[100049].0,Street,0.0,0,138,35281.0,50,"POLYGON ((13.45412 52.54035, 13.45320 52.53977..."
1,[100069498].0,Junction,0.0,0,200,44754.0,30,"POLYGON ((13.52273 52.50704, 13.52248 52.50690..."
2,"[100078509, 288268004, 3888645535].0",Junction,0.0,0,54,41983.0,50,"POLYGON ((13.47754 52.51457, 13.47782 52.51438..."


In [4]:
surface.head(3)

,id,type,score,incidents,rides,index_right,asphalt,concrete,paving_stone,sett,unpaved,geometry
0,[100049].0,Street,0.0,0,138,35281.0,0.000000,1.0,0.0,0.0,0.000000,"POLYGON ((13.45412 52.54035, 13.45320 52.53977..."
1,[100069498].0,Junction,0.0,0,200,44754.0,1.000000,0.0,0.0,0.0,0.000000,"POLYGON ((13.52273 52.50704, 13.52248 52.50690..."
2,"[100078509, 288268004, 3888645535].0",Junction,0.0,0,54,41983.0,0.846154,0.0,0.0,0.0,0.153846,"POLYGON ((13.47754 52.51457, 13.47782 52.51438..."


In [6]:
highway.head(3)

,id,type,score,incidents,rides,markers,index_right,highway,cycleway,footway,highway_rare,living_street,path,primary,residential,secondary,tertiary,track,geometry
0,[100049].0,Street,0.0,0,138,[ ],35281.0,"secondary, secondary, cycleway, secondary",0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.750000,0.0,0.0,"POLYGON ((13.45412 52.54035, 13.45320 52.53977..."
1,[100069498].0,Junction,0.0,0,200,[ ],44754.0,"residential, residential, residential",0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,"POLYGON ((13.52273 52.50704, 13.52248 52.50690..."
2,"[100078509, 288268004, 3888645535].0",Junction,0.0,0,54,[ ],41983.0,"secondary, secondary, cycleway, secondary, sec...",0.384615,0.0,0.0,0.0,0.0,0.0,0.153846,0.461538,0.0,0.0,"POLYGON ((13.47754 52.51457, 13.47782 52.51438..."


In [7]:
# Zusammenführen der ersten beiden GeoDataFrames
all_data = maxspeed.merge(surface, on=['id', 'geometry', 'type', 'score', 'incidents', 'rides', 'index_right'])

# Hinzufügen des dritten GeoDataFrames (highway)
all_data = all_data.merge(highway, on=['id', 'geometry', 'type', 'score', 'incidents', 'rides', 'index_right'])

In [8]:
all_data.head()

,id,type,score,incidents,rides,index_right,maxspeed,geometry,asphalt,concrete,...,cycleway,footway,highway_rare,living_street,path,primary,residential,secondary,tertiary,track
0,[100049].0,Street,0.0,0,138,35281.0,50,"POLYGON ((13.45412 52.54035, 13.45320 52.53977...",0.000000,1.0,...,0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.750000,0.000000,0.0
1,[100069498].0,Junction,0.0,0,200,44754.0,30,"POLYGON ((13.52273 52.50704, 13.52248 52.50690...",1.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0
2,"[100078509, 288268004, 3888645535].0",Junction,0.0,0,54,41983.0,50,"POLYGON ((13.47754 52.51457, 13.47782 52.51438...",0.846154,0.0,...,0.384615,0.0,0.0,0.0,0.0,0.0,0.153846,0.461538,0.000000,0.0
3,[100094].0,Street,0.0,0,98,31020.0,30,"POLYGON ((13.46855 52.61490, 13.46841 52.61475...",0.500000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0
4,[1000].0,Street,0.0,0,130,308.0,50,"POLYGON ((13.35533 52.51693, 13.35655 52.51683...",1.000000,0.0,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.666667,0.0


In [9]:
# Zeilen, die NaN  in 'index_right' enthalten
nan_values = all_data[all_data['index_right'].isna()]

print(f"Anzahl der Zeilen mit NaN in 'index_right': {len(nan_values)}")

Anzahl der Zeilen mit NaN in 'index_right': 0


In [11]:
#Speichern des gdf
output_filename = "../../data/processed_data/simra_osm_no_service_all.geojson"
all_data.to_file(output_filename, driver='GeoJSON')

print(f"Datei erfolgreich gespeichert.")

Datei erfolgreich gespeichert.
